**Write a better auto-complete algorithm using an N-gram model (similar models are used for translation, determining the author of a text, and speech recognition)**

In [ ]:
import random
import re
import nltk
from collections import defaultdict, Counter
from nltk.util import ngrams
from nltk.tokenize import word_tokenize

# Download NLTK tokenizer
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
def preprocess_text(text):
    """
    Cleans and tokenizes input text.
    """
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove punctuation
    tokens = word_tokenize(text)
    return tokens


In [ ]:
def train_ngram_model(text, n=2):
    """
    Trains an N-Gram model on input text and returns necessary data structures.
    """
    tokens = preprocess_text(text)
    vocab = set(tokens)
    unigram_counts = Counter(tokens)
    ngram_counts = defaultdict(Counter)

    # Create n-grams
    n_grams = list(ngrams(tokens, n, pad_left=True, pad_right=True, left_pad_symbol="<s>", right_pad_symbol="</s>"))

    # Count occurrences
    for ngram in n_grams:
        prefix = tuple(ngram[:-1])  # Previous words
        next_word = ngram[-1]       # Next word prediction
        ngram_counts[prefix][next_word] += 1

    return ngram_counts, unigram_counts, vocab

In [ ]:
def predict_next_word(text, ngram_counts, unigram_counts, vocab, n=2):
    """
    Predicts the most likely next word given a phrase.
    """
    tokens = preprocess_text(text)
    prefix = tuple(tokens[-(n - 1):])  # Get the last N-1 words as prefix

    # If prefix exists in n-grams, predict based on probability
    if prefix in ngram_counts:
        probable_words = ngram_counts[prefix]
        total_count = sum(probable_words.values())
        choices, weights = zip(*[(word, count / total_count) for word, count in probable_words.items()])
        return random.choices(choices, weights=weights)[0]

    # Fallback: Predict based on unigram probabilities
    elif unigram_counts:
        choices, weights = zip(*[(word, count / sum(unigram_counts.values())) for word, count in unigram_counts.items()])
        return random.choices(choices, weights=weights)[0]

    # Last fallback: Random word from vocabulary
    return random.choice(list(vocab)) if vocab else "No prediction available"

In [ ]:

text_corpus = """
Machine Learning is a subset of Artificial Intelligence that enables systems to learn and improve from experience.
Text prediction models are widely used in modern applications, including search engines and messaging platforms.
The n-gram approach is a fundamental technique in NLP for generating text-based predictions.
"""

ngram_counts, unigram_counts, vocab = train_ngram_model(text_corpus, n=2)  # Bigram Model

# Test Auto-complete
test_sentences = ["Machine Learning", "Text prediction", "The n-gram", "Artificial Intelligence is", "Systems to"]

print("\n=== N-Gram Auto-Complete Predictions ===\n")
for sentence in test_sentences:
    predicted_word = predict_next_word(sentence, ngram_counts, unigram_counts, vocab, n=2)
    print(f"Input: \033[1;34m'{sentence}'\033[0m → Predicted Next Word: \033[1;32m'{predicted_word}'\033[0m")
print("\n========================================\n")



=== N-Gram Auto-Complete Predictions ===

Input: 'Machine Learning' → Predicted Next Word: 'is'
Input: 'Text prediction' → Predicted Next Word: 'models'
Input: 'The n-gram' → Predicted Next Word: 'approach'
Input: 'Artificial Intelligence is' → Predicted Next Word: 'a'
Input: 'Systems to' → Predicted Next Word: 'learn'


